In [2]:
import pandas as pd
import csv
import psycopg2
from sqlalchemy import create_engine
import numpy as np


# Silver to Gold

In [3]:
#Final Cleaning from Silver to Aggregating for Gold 2018
# Repeat for each year
# Path to Silver Clean Files
clean_header_2018 = r'C:\Users\linnj\Desktop\SkillStorm\Projects\Project-1\Shipping Data\S3Layer=Silver\Year=2018\clean-2018\clean-header-2018.csv'
clean_hazmat_2018 = r'C:\Users\linnj\Desktop\SkillStorm\Projects\Project-1\Shipping Data\S3Layer=Silver\Year=2018\clean-2018\clean-hazmat-2018.csv'
clean_cargo_2018 = r'C:\Users\linnj\Desktop\SkillStorm\Projects\Project-1\Shipping Data\S3Layer=Silver\Year=2018\clean-2018\clean-cargo-2018.csv'

# Assign dataframes to read files
df1 = pd.read_csv(clean_header_2018, delimiter=',')
df2 = pd.read_csv(clean_hazmat_2018, delimiter=',')
df3 = pd.read_csv(clean_cargo_2018, delimiter=',')

# Drop automatically generated column 
df1.drop('Unnamed: 0', axis=1, inplace=True)
df2.drop('Unnamed: 0', axis=1, inplace=True)
df3.drop('Unnamed: 0', axis=1, inplace=True)

# Drop duplicate entries as safety check
df1.drop_duplicates(['Identifier', 'Identifier_Year'], inplace=True)
df2.drop_duplicates(['Identifier', 'Identifier_Year'], inplace=True)
df3.drop_duplicates(['Identifier', 'Identifier_Year'], inplace=True)

# Convert the Identifier year to float for Postgres readability
df1['Identifier_Year'] = df1['Identifier_Year'].astype(float)
df2['Identifier_Year'] = df1['Identifier_Year'].astype(float)
df3['Identifier_Year'] = df1['Identifier_Year'].astype(float)

# Aggregate tables and drop auto-generated duplicate columns
df4 = pd.merge(df1, df2, on='Identifier', how='outer')
df4.drop('Identifier_Year_y', axis=1, inplace=True)

# Aggregate tables and drop auto-generated duplicate columns
df6 = pd.merge(df2, df3, on='Identifier', how='outer')
df6.drop('Container_Number_y', axis=1, inplace=True)
df6.drop('Identifier_Year_y', axis=1, inplace=True)

# Aggregate final tables and drop auto-generated duplicate columns
df7 = pd.merge(df4, df6, on='Identifier', how='outer')
df7.drop(['Hazmat_Description_y', 'Hazmat_Code_Qualifier_y', 'Hazmat_Class_y', 'Hazmat_Code_y'], axis=1, inplace=True)
df7.drop('Container_Number_x', axis=1, inplace=True)
df7.drop('Identifier_Year_x_y', axis=1, inplace=True)
df7.rename(columns={"Identifier_Year_x_x": "Identifier_Year", "Hazmat_Code_x":"Hazmat_Code", 
"Hazmat_Class_x":"Hazmat_Class", "Hazmat_Code_Qualifier_x":"Hazmat_Code_Qualifier", 
"Hazmat_Description_x":"Hazmat_Description"}, inplace=True)

# Convert date columns to dateTime to allow for column aggregation, Postgres readability,
# and add column for difference between dates
df7.columns = df7.columns.str.lower()
df7['estimated_arrival_date'] = pd.to_datetime(df7['estimated_arrival_date'])
df7['actual_arrival_date'] = pd.to_datetime(df7['actual_arrival_date'])
df7['days_deviation'] = (df7['actual_arrival_date'] - df7['estimated_arrival_date']) / np.timedelta64(1, 'D')

# Set file path for Gold 2018 file and send to csv
gold_combined_2018 = r'C:\Users\linnj\Desktop\SkillStorm\Projects\Project-1\Shipping Data\S3Layer=Gold\UseCase=Hazmat_Shipment_Time\gold-combined-2018.csv'
df7.to_csv(gold_combined_2018)